In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

Here is a weird example where different col1 and col2 of data.json have different row number. We just want to show that different dimension is possible. In fact, a person can just request median(col_1), and median(col_2). Then just compute mean on his own as well, but here we show that the code is composable enough to do all at once. 

In [4]:

data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [5]:
scales = [5]
selected_columns = ['col_1', 'col_2']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x1, x2 = data[0], data[1]
    out_0 = s.median(x1)
    out_1 = s.median(x2)
    return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

error = 0.0001
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85214/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85214/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You ca

In [7]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path,selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85214/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_85214/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))


==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 10) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.003124237 | 0.006248474  | 0.006248474 | 0         | 0.003124237    | 0.006248474      | 0.006248474   | 0             | 0.000019521714     | 0.00006330774      | 0.00006330774          |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---

scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-6212,3220],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":20992,"total_assignments":20935,"total_const_size":3242,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[20992,[2]]},"required_lookups":[{"Floor":{"scale":2.0}},{"GreaterThan":{"a":0.0}},"ReLU"],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709716877850}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 3.5495471954345703 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.34375
==== Generating Proof ====
proof:  {'instances': [['02d91b6d36d8287fa18398dbe41fa396b3a2bd53aba2288c7b83866942c4ff1e', '19879c7f545b79f54649cf9692b2ace73fb0b0c526ee5251aef63bd64742df02', '0100000000000000000000000000000000000000000000000000000000000000', '2b06000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2c09c7217a6e198df0dcf3843ab416c31200bfa6e991c8c9b917283830c47dbe2d846ee55af934b08039154f749e09b111b5c5b4c1776d80e466a1e9e1e0b7042748c549088c372d16a2a54253f9cd9e1aa9a74d9ca4b29946fec6d7c16e83dd1b0ff077f88879d6e05fadcd8c207a6810a8b9b713a5d82a2b35cb1155bc5c2a10a51f0e6024fe4aa38cb22b2935288fe8a70aceea436a9dff15b3edb8c6b02a1d61eb7de4e1f2c1b5e0bfc604f9982718a9b2e701d7ad636217443740b3a8a4002197471f877287deb64065fd22897ac120223ece4e38bb63706eda402b89a00777af56a6be31b8b62bdff07ff6ac41b13ca62f7312c3c3d536d30cfa8e2230126b2ea

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [49.34375]
